In [60]:
from libraries import * 
from getData import * 

In [61]:
Augmentation_settings = {
    "Cross Validation Type": "kTkV",
    "model" : "SVR",
    "metric" : "all-metrics", # not required anymore

    "Augment" : False, # not implemented yet
    # "transform" : True, # need to implement this now. 

    "verbose" : 0,    # dont change this. Taken care of 
    "top_k" : 150,

    "repeat features" : False,
    "same split" :  False, #implement false condition
    
    "stratified" : False,
    "data" : "RS",
    
    "feature reduction" : "pearason",
    "features reduced per step" : 1
    # "order" : args.order if 'stratified' in args else None, #implement false condition
}

In [62]:
GET = getData(Augmentation_settings)
features, data, outputs = GET.data()

num_features = 150 

outputs = outputs.reshape(outputs.shape[0], 1)
data = np.hstack((data, outputs))
data = pd.DataFrame(data)

data.head()


loading resting state data
path =  /Users/saurav/Desktop/Margrit/MRI/RS/time_series


,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,-0.360065,-0.007469,0.054958,0.266475,0.232611,-0.338867,0.631779,-0.359282,-0.410621,-0.126857,...,0.102267,0.071445,0.572910,0.096260,0.013795,0.054751,-0.027064,0.266298,-0.134463,87.2
1,0.185745,0.159007,-0.080865,-0.160646,-0.047338,0.138317,0.461644,0.318556,0.088526,-0.203324,...,0.251529,-0.040121,0.591814,0.084357,0.088996,-0.136049,0.073134,0.295827,-0.101401,25.2
2,-0.000432,-0.106150,0.111955,0.033202,0.061812,0.090680,0.145437,-0.058595,-0.129418,-0.399258,...,-0.148758,0.048900,0.243912,-0.075932,0.037224,0.024014,0.164834,0.163829,0.143013,52.0
3,-0.066249,0.105581,-0.195710,-0.257786,-0.055454,-0.130789,0.770839,0.209555,-0.084566,-0.312965,...,0.302855,0.430871,0.405293,0.045807,0.209190,0.094675,-0.078121,0.682280,0.009166,74.1
4,0.047168,0.040613,-0.066341,-0.095488,-0.215614,0.012098,0.140075,0.207200,-0.067715,-0.029938,...,0.257647,0.006619,0.504215,-0.046846,-0.000666,-0.150161,-0.164306,0.597433,-0.112093,30.8


In [63]:
# data.columns = list(features) + ["outputs"]
corrs = data.corr().abs()
importances = corrs.values[-1][:-1]

top_columns = np.argpartition(importances, -num_features)[-num_features:]
corrs


,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1.000000,0.231838,0.105004,0.206959,0.267749,0.631039,0.194382,0.480949,0.642868,0.043913,...,0.142265,0.118421,0.036892,0.009484,0.172658,0.191546,0.044175,0.031991,0.075524,0.011537
1,0.231838,1.000000,0.290545,0.025862,0.222755,0.163703,0.122384,0.238191,0.320355,0.171362,...,0.164165,0.108097,0.010622,0.001557,0.024834,0.133421,0.157720,0.271320,0.135513,0.273330
2,0.105004,0.290545,1.000000,0.684147,0.515099,0.118176,0.234398,0.063864,0.027732,0.428390,...,0.212825,0.147968,0.065598,0.242674,0.019615,0.260638,0.141741,0.012120,0.005646,0.033094
3,0.206959,0.025862,0.684147,1.000000,0.423187,0.321434,0.221016,0.326620,0.243296,0.169183,...,0.205537,0.074396,0.022891,0.175307,0.075826,0.006107,0.045449,0.178734,0.058090,0.048156
4,0.267749,0.222755,0.515099,0.423187,1.000000,0.225265,0.024054,0.221619,0.124094,0.067933,...,0.049686,0.218340,0.337795,0.068941,0.014292,0.143202,0.130254,0.075702,0.237466,0.110672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,0.191546,0.133421,0.260638,0.006107,0.143202,0.227571,0.025672,0.209403,0.095522,0.235482,...,0.178394,0.222179,0.336911,0.308112,0.126345,1.000000,0.198749,0.242173,0.422373,0.182413
1125,0.044175,0.157720,0.141741,0.045449,0.130254,0.261334,0.049302,0.031380,0.216983,0.025018,...,0.014361,0.233683,0.066077,0.263634,0.142017,0.198749,1.000000,0.310756,0.574969,0.171765
1126,0.031991,0.271320,0.012120,0.178734,0.075702,0.072916,0.134682,0.118834,0.142704,0.248029,...,0.254972,0.113076,0.009578,0.274958,0.018482,0.242173,0.310756,1.000000,0.100227,0.180631
1127,0.075524,0.135513,0.005646,0.058090,0.237466,0.206501,0.084420,0.196309,0.141085,0.230036,...,0.083791,0.452231,0.188752,0.270047,0.192889,0.422373,0.574969,0.100227,1.000000,0.134994


In [64]:
np.argmax(corrs.values[-1][:-1])

506

In [65]:
import heapq

test_top_cols = heapq.nlargest(150, range(len(importances)), importances.take)

test_top_cols.sort()
top_columns.sort()
# test_top_cols

## Training loop test

In [66]:
outputs = outputs.flatten()
data.head()



,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,-0.360065,-0.007469,0.054958,0.266475,0.232611,-0.338867,0.631779,-0.359282,-0.410621,-0.126857,...,0.102267,0.071445,0.572910,0.096260,0.013795,0.054751,-0.027064,0.266298,-0.134463,87.2
1,0.185745,0.159007,-0.080865,-0.160646,-0.047338,0.138317,0.461644,0.318556,0.088526,-0.203324,...,0.251529,-0.040121,0.591814,0.084357,0.088996,-0.136049,0.073134,0.295827,-0.101401,25.2
2,-0.000432,-0.106150,0.111955,0.033202,0.061812,0.090680,0.145437,-0.058595,-0.129418,-0.399258,...,-0.148758,0.048900,0.243912,-0.075932,0.037224,0.024014,0.164834,0.163829,0.143013,52.0
3,-0.066249,0.105581,-0.195710,-0.257786,-0.055454,-0.130789,0.770839,0.209555,-0.084566,-0.312965,...,0.302855,0.430871,0.405293,0.045807,0.209190,0.094675,-0.078121,0.682280,0.009166,74.1
4,0.047168,0.040613,-0.066341,-0.095488,-0.215614,0.012098,0.140075,0.207200,-0.067715,-0.029938,...,0.257647,0.006619,0.504215,-0.046846,-0.000666,-0.150161,-0.164306,0.597433,-0.112093,30.8


In [72]:

def important_columns(data, outputs, num_features): 
    """This function finds the names of important features in the data, either based on two factors
        - Pearson Correlation with WAB scores 
        - Recurrent Feature Elimination (RFE)

    Args:
        data (numpy array): Input data without output column
        outputs (numpy array): 1D array of WAB scores / any other output

    Returns:
        numpy array : finds the column index numbers and returns it 
    """    

    outputs = outputs.reshape(outputs.shape[0], 1)
    data = np.hstack((data, outputs))
    data = pd.DataFrame(data)

    corrs = data.corr().abs()
    importances = corrs.values[-1][:-1]

    top_columns = np.argpartition(importances, -num_features)[-num_features:]
    return top_columns

def reduce_data( data, top_columns):
    """filter important columns from the input data

    Args:
        data (numpy array): Input data without output columns

    Returns:
        numpy array: Input data with selected top features
    """
    
    data = pd.DataFrame(data)
    data = data.iloc[:,top_columns]
    return data.values
    


In [132]:
# data = data.values
# data = data[:,:-1]
data2 = data

In [135]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel



kf = KFold(n_splits=11, shuffle = True)

num_features = 25
total , count = 0,0

params = {
    "n_estimators": 500,
    "max_depth": 30,
    "min_samples_split": 2,
    "learning_rate": 0.00001,
    "loss": "ls",
}
top_columns = important_columns(data, outputs, num_features)
X_train = reduce_data(data, top_columns)

for train_index, validate_index in kf.split(data, outputs):
    X_train, X_test, y_train, y_test = data[train_index], data[validate_index], outputs[train_index], outputs[validate_index] 
    # top_columns = important_columns(X_train, y_train, num_features)
    # X_train = reduce_data(X_train, top_columns)
    # X_test = reduce_data(X_test, top_columns)
 
    # kernel = DotProduct() + WhiteKernel()
    
    # m1 = GaussianProcessRegressor(kernel=kernel, random_state=0)
    # m1 = ensemble.GradientBoostingRegressor(**params)   
    # m1 = AdaBoostRegressor(RandomForestRegressor(max_depth=5, random_state=0))
    
    m1 = AdaBoostRegressor(SVR(kernel = "linear", C=1, epsilon=10, tol=0.01, gamma=1e-3))
    model = make_pipeline(StandardScaler(), m1)
    model.fit(X_train, y_train)
    
    test_outputs = model.predict(X_test)
    mse = mean_squared_error(test_outputs, y_test, squared=False)
    
    total+=mse 
    count+=1
    
    print(test_outputs, " -- ", mse)
    # print(y_test)

print("mean rmse = ", float(total/count))



[62.8775271  54.88075783 57.47601334 59.65109178 69.44145882]  --  21.441909466093673
[53.46994776 57.41393091 65.89395901 68.84652212 69.37231472]  --  21.521391759865427
[62.01827608 60.58096351 67.89312279 68.89568014 69.38485639]  --  28.89929220535707
[69.24179185 63.92448064 62.98333279 65.3905103  70.14190337]  --  28.975892760547104
[75.99682532 67.75055461 69.39321837 82.03643542 59.97585907]  --  17.316080966391137
[59.0003309  62.6137666  50.97141126 81.02389895 71.55495703]  --  24.40554859589116
[65.49623452 64.95465626 70.10976615 65.15957619 63.91266977]  --  23.374218340396485
[67.50033243 72.92022438 73.15172227 66.69538247 79.690335  ]  --  19.427007203423297
[56.8182744  67.93612582 75.36565606 68.02549464 58.68802928]  --  16.678809835741774
[71.37754764 70.49590017 72.43324831 76.37256812 67.41045608]  --  29.559888127231616
[70.84167949 66.96301448 64.77157511 72.48024486 59.95053366]  --  13.836128534346118
mean rmse =  22.312378890480442
